In [1]:
%run Model.ipynb

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c
nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c
accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c
four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c
four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c
9.605894736842117
Uptake: -10.0
Yield for model SucD = 0.9605894736842118
akg_c --> co2_c + succinyl_semialdehyde_c
nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c
accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c
four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c
four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c
9.708085106382972
Uptake: -10.0
Yield for model Akg = 0.9708085106382972


# Co-Factor Swap attempt 

Most of the metabolic processes rely on the functionality of the co-factors. By swapping the co-factors we have the ability to increase the production of our product or a target metabolite. At the same time, the growth is being also affected by the co-factor swapping. Lastly, the co-factor swapping can also optimize an already existing pathway.

In [2]:
from cameo import models
model_orig = model_sucd #importing our model

In [3]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [4]:
model = model_orig.copy() # create a copy for changing the metobolites and reactions

## Biomass yield and co-factor swap model

Here we calculate the biomass yield of our product in order to be compared with the swapped one at the very end. Also, the code for the co-factor is presented. The code is changing the co-factors with the lowest yield and replace them with the co-factors with the highest yield, and saves them to the initial model. 

In [5]:
solution = model.optimize()
g_dw_biomass_per_mmol_glucose = solution.objective_value / (-1. * solution.fluxes['EX_glc__D_e'])
print ('Biomass yield=', g_dw_biomass_per_mmol_glucose)

Biomass yield= 0.08769972144269841


In [6]:
for rid in ['SUCD', 'FHBD', 'HBCT', 'HBCR','ADH' ]: # we insert our reactions in our model
    
    model.reactions.get_by_id(rid).knock_out() #checking the reactions and replace them with a better yield
model.reactions.ADH.lower_bound = 0
model.reactions.EX_glc__D_e.lower_bound = -10
model.reactions.EX_o2_e.lower_bound = -10


In [7]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.1 # we set a lower bounder for the biomass

## Calculation of yield in cmol

In this part, the yield in cmol is calculated in order to see the yield per single carbon molecule. 


In [8]:
model.objective = model.reactions.EX_glc__D_e
(model.optimize().objective_value * 4) / (model.reactions.EX_glc__D_e.flux * 6) #the yield is calculated in cmol 

0.6666666666666666

## Swap analysis for identifying targets 

In this part is being attempted the swap analysis for the identification of potential targets.

In [13]:
py = product_yield(model.reactions.ADH, model.reactions.EX_glc__D_e) # product's yield
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [14]:
optswap.run(max_evaluations=200, max_size=2) # No results of cofactor swap 

Starting optimization at Fri, 26 Nov 2021 18:05:12


HBox()

Finished after 00:00:30


,index,targets,fitness


In [15]:
# Calculations of yields before and after the swap

In [16]:
#Here are the calculations before and after the co-factor swap in order to compare the yields of the final product in both cases, respectively. 

In [17]:
#optswap.model.reactions.ADH.model = optswap.model
#optswap.model.objective = optswap.model.reactions.ADH
#original = (optswap.model.optimize().objective_value * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6) # calculaates the yield before the substrate swap
#with TimeMachine() as tm:
#    optswap.model.swap_reaction('ADH', tm)
#    swapped = (optswap.optimize().objective_value * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6) # calculaates the yield after the substrate swap

#print("product/substrate yield without swap: {}\nproduct/substrate yield with swap: {}".format(original, swapped))

## Co Factor Swap

In this project, an attempt to switch the co-factors was provided. Recent research has found that modifying the co-factors in a certain route almost always results in an increase in the final product yield (Armingol, 2018), as well as having an effect on the maximum theoretical yield by increasing it (Kinga, 2014). However, because certain enzymes are required to generate specific compounds, the co-factor swap is not always effective. As a result, the co-factor swap approach was adjusted and implemented in this project for the generation of 1,4-BDO. However, after running the algorithm, there were no components that needed to be exchanged as it is depicted in Figure X. Although, investigating the factor's concentrations on each stage is another approach to pursue the co-factor swap model results actively. By analyzing the concentrations of our response variables on both SUCD and AKG models at the same time. 

<img src="Pictures/Co_factor result.png" width="700" style="margin:auto" align="center"/>

<p style="text-align: center;">Figure X. Outcome from the co-factor algorithm performance.</p>


XX. Armingol E, TobarE, CabreraR (2018)Understanding the impactof the cofactorswappingof isocitratedehydrogenase over the growthphenotypeofEscherichiacolion acetate by usingconstraint-basedmodeling. PLoSONE 13(4):e0196182.https://doi.org/10.1371/journal.pone.0196182

XX. Zachary A.Kinga and Adam M.Feistab, Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae, Metabolic Engineering, vol 24, July 2014, Pages 117-128. https://doi.org/10.1016/j.ymben.2014.05.009